In [1]:
import rasterio
from rasterio.enums import Resampling
from rasterio.dtypes import uint8, float64
from pathlib import Path
import numpy as np
from rasterio.crs import CRS

In [2]:
%%time
import rasterio.warp

EPSG2193 = CRS.from_epsg(2193)

CPU times: user 4.56 ms, sys: 68 µs, total: 4.63 ms
Wall time: 4.73 ms


In [3]:
def align(output, files, profile, transform, nodata=0, dtype=uint8,resampling=Resampling.bilinear):
    with rasterio.open(output, 'w', **profile) as dst:
        for band, f in enumerate(files):
            with rasterio.open(f, 'r') as src:
                data, transform = rasterio.warp.reproject(
                    source=src.read(1),
                    destination=np.zeros((1, 100, 100), dtype),
                    src_transform=src.transform,
                    dst_transform=transform,
                    src_crs=EPSG2193,
                    dst_crs=EPSG2193,
                    dst_nodata=nodata,
                    resampling=resampling
                )
                dst.write(data[0], band+1)

In [4]:
%%time
discrete_files = list(sorted(Path('discrete').glob('*.asc')))

# Take the first raster as a template
with rasterio.open(discrete_files[0], 'r') as src:
    src_transform = src.transform
    src_profile = src.profile

src_profile.update(driver='GTiff') # Upgrade from ASC to GeoTIFF
src_profile.update(crs=EPSG2193) # Declare (missing) EPSG

src_profile.update(count=len(discrete_files)) # Make room for all bands
align(
    Path('discrete.tif'),
    discrete_files,
    src_profile,
    src_transform,
    nodata=np.iinfo(uint8).max,
    dtype=uint8,
    resampling=Resampling.nearest
)

CPU times: user 37 s, sys: 3.55 s, total: 40.5 s
Wall time: 40.6 s


In [5]:
%%time
continuous_files = list(sorted(Path('continuous').glob('*.asc')))

# Take the first raster as a template
with rasterio.open(continuous_files[0], 'r') as src:
    src_transform = src.transform
    src_profile = src.profile

src_profile.update(driver='GTiff') # Upgrade from ASC to GeoTIFF
src_profile.update(crs=EPSG2193) # Declare (missing) EPSG

src_profile.update(dtype=float64)
src_profile.update(count=len(continuous_files)) # Make room for all bands
align(
    Path('continuous.tif'),
    continuous_files,
    src_profile,
    src_transform,
    nodata=np.finfo(float64).max,
    dtype=float64
)

CPU times: user 49.4 s, sys: 4.65 s, total: 54.1 s
Wall time: 59.5 s
